In [22]:
import numpy as np
from fluid import Fluid
import scipy as sp
from scipy import optimize as op
from scipy.optimize import minimize

import json

In [34]:
def netPow(x):
        
    m_dot_c = x[0]
    P_pump_discharge = x[1]

    #Givens
    m_dot_s = 3.2 # mass flow of the continuous supply being sequestered (s for sequestration)
    #State 5 is the incoming sequestration state 
    T5 = 239.15 
    P5 = 6.0 

    #State 1
    m_dot_1 = m_dot_s + m_dot_c
    T1 = T5 # to match sequestration state
    P1 = P5 #to match sequestration state

    co2 = Fluid('co2')
    s1 = co2.entropy(T=T1, P=P1)
    h1 = co2.enthalpy(T = T1, P = P1)

    #State 2
    m_dot_2 = m_dot_s + m_dot_c
    s2 = s1 #Pump modeled as isentropic
    P2 = P_pump_discharge #Free variable

    h2 = co2.enthalpy(s = s2, P = P2)
    T2 = co2.temperature(s = s2, P = P2)

    pow_in = m_dot_2 * (h2 - h1)

    #State 3
    dP=0 #for now
    dT=0 #for now
    m_dot_3 = m_dot_c
    P3 = P2 - dP #dP is the pressure loss of the underground system
    T3 = T2 + dT #dT is the temperature increase after the underground system

    s3 = co2.entropy(T=T3, P=P3)
    h3 = co2.enthalpy(T = T3, P = P3)

    #State 4
    m_dot_4 = m_dot_c
    s4 = s3 #Turbine modeled as isentropic
    P4 = P1 # Condenser modeled as isobarric

    h4 = co2.enthalpy(s = s4, P = P4)
    pow_out = m_dot_4 * (h3 - h4)
    
    net_pow = -1 * (pow_out - pow_in) 
    return net_pow

# Initial guess for the optimization
initial_guess = [2, 20]

# Call minimize function
result = op.minimize(netPow, initial_guess)
 
# Extract the optimal values
optimal_m_dot_c, optimal_P_pump_discharge = result.x
print(optimal_m_dot_c)
print(optimal_P_pump_discharge)
# You can also get the minimum value of netPow
minimum_net_pow = result.fun

print(minimum_net_pow)

#t = op.minimize_scalar(net_pow, [param_dict['T_inf'], T_l_init, 275], bounds=(0, np.inf))

ValueError: Error in calculating property: Input pair variable is invalid and output(s) are non-trivial; cannot do state update : PropsSI("H","s",4399.244053,"P",20,"co2")